# Blondy's Soaring Prediction System

In [81]:
import json
import os
import shutil
from herbie import Herbie
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from herbie.toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd
import matplotlib as mpl

pd.set_option('display.max_rows', 500)

recent = pd.Timestamp.utcnow().floor("6h").tz_localize(None) #- pd.Timedelta("6h")
recent

Timestamp('2024-12-09 00:00:00')

In [2]:
data_dir = "/Users/andrew/data/hrdps/"
#fig_dir = "/Users/andrew/Repos/BMWF/plots/"


In [10]:
# read in the config
with open('bsp_config.json', 'r') as f:
    config = json.load(f)

1) set up templates for each location

2) grab a slice, punch out the locations, merge into template

3) create a time-series for each level at a location, then concatenate the levels to generate a column model

4) save each location dataset, purge the raw data directory

In [77]:
# make a list of the variables we need and at which levels
df = pd.DataFrame(columns=["variable", "level"])

for variable in config["variables"]:
    for level in config["levels"][config["variables"][variable]]:
        _df = pd.DataFrame({"variable":[variable], "level":[level]})
        df = pd.concat([df, _df]) 
        
df = df.reset_index(drop=True)

# get indices of each variable
list(df[df["variable"] == "TMP"].index)

[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]

In [68]:
# set up templates for each location
locations = {loc: [None]*len(df) for loc in config["locations"].keys()}
print(locations["revelstoke"][0])

None


In [82]:
# grab a slice
ds = Herbie(recent,
            model="hrdps",
            fxx=0,
            product="continental/2.5km",
            variable=df["variable"].iloc[0],
            level=df["level"].iloc[0]
           ).xarray()

# if the variable name is unknown, assign it
if "unknown" in ds:
    ds = ds.rename({"unknown":df["variable"].iloc[0]})
    
# punch out locations
for loc in config["locations"]:
    locations[loc][0] = ds.sel((((ds.longitude - config["locations"][loc]["longitude"]) ** 2 
                        + (ds.latitude - config["locations"][loc]["latitude"]) ** 2) ** 0.5).argmin(dim=["x", "y"]))

# purge the data directory
for filename in os.listdir(data_dir):
    file_path = os.path.join(data_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ local ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")


In [84]:
locations["revelstoke"]

[<xarray.Dataset> Size: 60B
 Dimensions:              ()
 Coordinates:
     time                 datetime64[ns] 8B 2024-12-09
     step                 timedelta64[ns] 8B 00:00:00
     surface              float64 8B 0.0
     latitude             float64 8B 50.96
     longitude            float64 8B -118.1
     valid_time           datetime64[ns] 8B ...
     gribfile_projection  object 8B None
 Data variables:
     TCDC                 float32 4B ...
 Attributes:
     GRIB_edition:            2
     GRIB_centre:             cwao
     GRIB_centreDescription:  Canadian Meteorological Service - Montreal
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             Canadian Meteorological Service - Montreal
     model:                   hrdps
     product:                 continental/2.5km
     description:             Canada's High Resolution Deterministic Predictio...
     remote_grib:             /Users/andrew/data/hrdps/20241209/20241209T00Z_M...
  

In [ ]:
[variable for variable in config["variables"] if config["variables"][variable] == "ISBL"]

In [ ]:
_ds = Herbie(recent,
             model="hrdps",
             fxx=1,
             product="continental/2.5km",
             variable="HGT",
             level="ISBL_0500"
            ).xarray()

In [ ]:
_ds

In [ ]:
df.reset_index(drop=True)